In [ ]:
# Install required packages if not already installed
# !pip install dash dash-bootstrap-components pandas plotly sqlalchemy

import pandas as pd
import sqlite3
from dash import Dash, dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
from io import BytesIO
import base64

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

In [ ]:
conn = sqlite3.connect('customer360.db', check_same_thread=False)


In [ ]:
app.layout = dbc.Container([
    html.H1("Customer 360° Analytics Dashboard", style={'textAlign': 'center', 'marginTop': 20}),

    dbc.Row([
        dbc.Col([
            dcc.Upload(
                id='upload-data',
                children=html.Div(['Drag and Drop or ', html.A('Select CSV File')]),
                style={'width': '100%', 'height': '60px', 'lineHeight': '60px',
                       'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                       'textAlign': 'center', 'marginBottom': 20},
                multiple=True
            ),
            dbc.Button("Refresh KPIs & Graphs", id="refresh-kpi", color="primary", className="mb-2"),
            dbc.Button("Add New Row", id="add-row", color="success", className="mb-2", style={'marginLeft':'10px'}),
            dbc.Button("Download Master Table", id="download-btn", color="info", className="mb-2", style={'marginLeft':'10px'}),
            dcc.Download(id="download-dataframe-csv")
        ], width=4),

        dbc.Col([
            html.H4("KPIs"),
            html.Div(id='kpi-output')
        ], width=8)
    ]),

    dbc.Row([
        dbc.Col([
            html.H4("Master Table (Editable)"),
            dash_table.DataTable(
                id='editable-table',
                columns=[],
                data=[],
                editable=True,
                row_deletable=True,
                filter_action="native",
                sort_action="native",
                page_size=10
            )
        ], width=12)
    ], style={'marginTop': 20, 'marginBottom': 20}),

    dbc.Row([
        dbc.Col([
            html.Label("Filter by Age Group:"),
            dcc.Dropdown(id='filter-age', multi=True)
        ], width=4),
        dbc.Col([
            html.Label("Filter by Churned Status:"),
            dcc.Dropdown(id='filter-churn', multi=True)
        ], width=4)
    ], style={'marginBottom': 20}),

    dbc.Row([
        dbc.Col([dcc.Graph(id='churn-graph')], width=6),
        dbc.Col([dcc.Graph(id='retention-graph')], width=6),
    ]),

    dbc.Row([
        dbc.Col([dcc.Graph(id='segmentation-graph')], width=12)
    ]),

    html.Div(id='master-data', style={'display': 'none'})
], fluid=True)


In [ ]:
def parse_csv(contents, filename):
    import base64, io
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        if filename.endswith('.csv'):
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            return df
    except Exception as e:
        print(e)
        return pd.DataFrame()

def create_master_table(df_list):
    if len(df_list) == 0:
        return pd.DataFrame()
    master_df = df_list[0]
    for df in df_list[1:]:
        if 'customer_id' in df.columns and 'customer_id' in master_df.columns:
            master_df = master_df.merge(df, on='customer_id', how='outer')
        else:
            master_df = pd.concat([master_df, df], axis=0, ignore_index=True)
    if 'churned' in master_df.columns:
        master_df['churned'] = master_df['churned'].fillna(0).astype(int)
    if 'age' in master_df.columns:
        master_df['age'] = master_df['age'].fillna(master_df['age'].median())
    return master_df

In [ ]:
@app.callback(
    Output('master-data', 'children'),
    Output('editable-table', 'columns'),
    Output('editable-table', 'data'),
    Output('filter-age', 'options'),
    Output('filter-churn', 'options'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_master(contents, filenames):
    if contents is not None:
        dfs = [parse_csv(c, f) for c, f in zip(contents, filenames)]
        master_df = create_master_table(dfs)
        master_df.to_sql('master_table', conn, if_exists='replace', index=False)
        columns = [{"name": i, "id": i, "editable": True} for i in master_df.columns]
        data = master_df.to_dict('records')
        age_options = [{"label": str(a), "value": str(a)} for a in sorted(master_df['age'].astype(int).unique())] if 'age' in master_df.columns else []
        churn_options = [{"label": str(c), "value": str(c)} for c in sorted(master_df['churned'].unique())] if 'churned' in master_df.columns else []
        return master_df.to_json(date_format='iso', orient='split'), columns, data, age_options, churn_options
    return pd.DataFrame().to_json(date_format='iso', orient='split'), [], [], [], []

@app.callback(
    Output('master-data', 'children', allow_duplicate=True),
    Input('editable-table', 'data'),
    prevent_initial_call=True
)
def update_from_table(table_data):
    df = pd.DataFrame(table_data)
    df.to_sql('master_table', conn, if_exists='replace', index=False)
    return df.to_json(date_format='iso', orient='split')

@app.callback(
    Output('editable-table', 'data', allow_duplicate=True),
    Input('add-row', 'n_clicks'),
    State('editable-table', 'columns'),
    State('editable-table', 'data'),
    prevent_initial_call=True
)
def add_new_row(n_clicks, columns, data):
    new_row = {c['id']: "" for c in columns}
    data.append(new_row)
    return data

@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-btn", "n_clicks"),
    State('master-data', 'children'),
    prevent_initial_call=True
)
def download_csv(n_clicks, master_json):
    df = pd.read_json(master_json, orient='split')
    return dcc.send_data_frame(df.to_csv, "master_table.csv", index=False)

@app.callback(
    Output('kpi-output', 'children'),
    Input('refresh-kpi', 'n_clicks'),
    State('master-data', 'children')
)
def update_kpis(n, master_json):
    df = pd.read_json(master_json, orient='split')
    if df.empty:
        return "No data available."
    total_customers = df['customer_id'].nunique() if 'customer_id' in df.columns else len(df)
    churn_rate = round(df['churned'].mean() * 100, 2) if 'churned' in df.columns else 0
    avg_age = round(df['age'].mean(), 2) if 'age' in df.columns else 0
    color_churn = "red" if churn_rate > 50 else "green"
    return dbc.Row([
        dbc.Col(dbc.Card([dbc.CardHeader("Total Customers"), dbc.CardBody(html.H4(total_customers))], color="primary", inverse=True)),
        dbc.Col(dbc.Card([dbc.CardHeader("Churn Rate"), dbc.CardBody(html.H4(f"{churn_rate}%"))], color=color_churn, inverse=True)),
        dbc.Col(dbc.Card([dbc.CardHeader("Average Age"), dbc.CardBody(html.H4(avg_age))], color="info", inverse=True))
    ])

In [ ]:
@app.callback(
    Output('churn-graph', 'figure'),
    Output('retention-graph', 'figure'),
    Output('segmentation-graph', 'figure'),
    Input('master-data', 'children'),
    Input('filter-age', 'value'),
    Input('filter-churn', 'value')
)
def update_graphs(master_json, age_filter, churn_filter):
    df = pd.read_json(master_json, orient='split')
    if df.empty:
        return {}, {}, {}
    # Apply filters
    if age_filter and 'age' in df.columns:
        df = df[df['age'].astype(str).isin(age_filter)]
    if churn_filter and 'churned' in df.columns:
        df = df[df['churned'].astype(str).isin(churn_filter)]

    churn_fig = px.pie(df, names='churned', title="Churn Distribution") if 'churned' in df.columns else {}
    if 'age' in df.columns:
        df['age_group'] = pd.cut(df['age'], bins=[0,18,25,35,50,100], labels=['0-18','19-25','26-35','36-50','50+'])
        retention_fig = px.bar(df.groupby('age_group')['customer_id'].count().reset_index(), x='age_group', y='customer_id', title="Customer Retention by Age Group")
    else:
        retention_fig = {}
    if 'age_group' in df.columns and 'churned' in df.columns:
        segmentation_fig = px.histogram(df, x='age_group', color='churned', barmode='group', title="Customer Segmentation by Age Group & Churn")
    else:
        segmentation_fig = {}
    return churn_fig, retention_fig, segmentation_fig

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>